In [2]:
import tensorflow as tf
import pennylane as qml
import numpy as np
import sys
from qGAN import qGAN, train_qGAN

Here we will train the qGAN created in the file qGAN.py and train it on some simple examples.
First, a function to create arbitrary adjacency matrices.

In [2]:
def random_adjacency_matrix(n):
    """
    creates an nxn symmetric adjacency matrix with 0s along the diagonal
    used to represent an undirected graph with n nodes
    :param n: dimension
    :return: nxn numpy array
    """
    matrix = [[np.random.randint(0, 1) for i in range(n)] for j in range(n)]
    # No vertex connects to itself
    for i in range(n):
        matrix[i][i] = 0
    # If i is connected to j, j is connected to i
    for i in range(n):
        for j in range(n):
            matrix[j][i] = matrix[i][j]
    return np.array(matrix)

In [3]:
dataset = [random_adjacency_matrix(3) for _ in range(50)]
train_qGAN(dataset[0], dataset, 30)




Instructions for updating:
Use tf.identity instead.


Instructions for updating:
Use tf.identity instead.


Gen cost: [4.75834439]
Disc cost: [-0.17832228]
Generated sample:
[[1. 1. 1.]
 [0. 0. 1.]
 [1. 1. 1.]]
Gen cost: [1.68520967]
Disc cost: [-0.68732201]
Generated sample:
[[0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Gen cost: [0.14393633]
Disc cost: [-0.90785427]
Generated sample:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Gen cost: [0.03072745]
Disc cost: [-0.98380213]
Generated sample:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Gen cost: [0.00508255]
Disc cost: [-0.9993631]
Generated sample:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Gen cost: [0.00369175]
Disc cost: [-0.99998843]
Generated sample:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


Penalties: We are getting lots of matrices with 1 on the diagonal, which we ideally don't want. We also want to penalise going to a  location twice. This means we want to penalise measurements of qubits 0, n+1, 2n+2, ..., n**2 - 1.
This is just the Z expectation value of these qubits.

We also require that the qubits in each row and column have hamming weight 1. These qubits are $|0, 1, ..., n \rangle$, $|n + 0, n + 1, ..., n \rangle$, ... $|n^2 - n, ..., n^2 \rangle$.

And for the columns we have $|n.0, n.1, ..., n(n-1) \rangle$, $|n.0 + 1, n.1 + 1, ..., n(n-1) + 1 \rangle$, $|n.0 + n, n + n, ..., n(n-1) + n\rangle$.

In [ ]:
#Training on data from DWave
samples = np.load('../AirlineDataSets/4_qubit_adj_mat.npy')
s = [x for x in samples]
output = train_qGAN(s[0], s, 50, gen_dev='qiskit.aer', disc_dev='qiskit.aer')

In [13]:
def stitch_sparse(in_matrix, out_matrix):
    n = in_matrix.shape[0]
    in_sparse = []
    out_sparse = []
    for i in range(n):
        for j in range(n):
            if (in_matrix[i][j] > 0):
                in_sparse.append([i,j])
            if (out_matrix[i][j] > 0):
                out_sparse.append([i,j])
    in_sparse, out_sparse = np.array(in_sparse), np.array(out_sparse)
    return np.concatenate((in_sparse,out_sparse),axis=1)

In [14]:
def decompose_sparse(in_sparse):
    decomp = np.hsplit(in_sparse, 2)
    n = in_sparse.shape[0]
    count = 0
    in_matrix = []
    out_matrix = []
    for i in range(int(n/2)):
        for j in range(n):
            if count < n:
                in_matrix.append(decomp[i][j])
            else:
                out_matrix.append(decomp[i][j])
            count += 1
    in_matrix, out_matrix = np.array(in_matrix), np.array(out_matrix)
    return in_matrix, out_matrix

In [16]:
ma = np.load('../AirlineDataSets/16qbMa.npy')
mb = np.load('../AirlineDataSets/16qbMb.npy')
inputs = [stitch_sparse(a, b) for a, b in zip(ma, mb)]
output = train_qGAN(inputs[1], inputs, 15, gen_dev='qiskit.aer', disc_dev='qiskit.aer')
samples = [decompose_sparse(x) for x in output]
print(samples)

ValueError: cannot reshape array of size 12 into shape (9,)